# Genetic Heritage

For this assignment, we are given the following strings

In [98]:
strings =  [(0,'TTCTACGGGGGGAGACCTTTACGAATCACACCGGTCTTCTTTGTTCTAGCCGCTCTTTTTCATCAGTTGCAGCTAGTGCATAATTGCTCACAAACGTATC'),
            (1,'TCTACGGGGGGCGTCATTACGGAATCCACACAGGTCGTTATGTTCATCTGTCTCTTTTCACAGTTGCGGCTTGTGCATAATGCTCACGAACGTATC'),
            (2,'TCTACGGGGGGCGTCTATTACGTCGCCAACAGGTCGTATGTTCATTGTCATCATTTTCATAGTTGCGGCCTGTGCGTGCTTACGAACGTATTCC'),
            (3,'TCCTAACGGGTAGTGTCATACGGAATCGACACGAGGTCGTATCTTCAATTGTCTCTTCACAGTTGCGGCTGTCCATAAACGCGTCCCGAACGTTATG'),
            (4,'TATCAGTAGGGCATACTTGTACGACATTCCCCGGATAGCCACTTTTTTCCTACCCGTCTCTTTTTCTGACCCGTTCCAGCTGATAAGTCTGATGACTC'),
            (5,'TAATCTATAGCATACTTTACGAACTACCCCGGTCCACGTTTTTCCTCGTCTTCTTTCGCTCGATAGCCATGGTAACTTCTACAAAGTTC'),
            (6,'TATCATAGGGCATACTTTTACGAACTCCCCGGTGCACTTTTTTCCTACCGCTCTTTTTCGACTCGTTGCAGCCATGATAACTGCTACAAACTTC')]

## Overview

These were generated by taking one existing string, and applying some number of transformations (switching, adding, or deleting a character), each done with a low probability. One string was the base, with which two children were created (using transformations). Each of these two children then had two children. We are left with seven strings, but we don't know the lineage. Our task is to find the lineage of these strings. We know that the changes are unlikely, and thus the lineage with the least changes overall (amongst all parents and children) is the most likely lineage.

Before we begin our analysis, we will more explicitly review the given information about the strings. We know several things about the strings. Firstly, there are 7 strings, one grandparent, 2 children, and 4 grandchildren. Secondly, the possible operations are inserting a new charecter, deleting an existing charecter, and changing a character. The first metric we will examine them on is LCS, or longest common subsequence.

If we have our initial string : TTCTACGGGG
and we insert a new character anywhere in the string : TTCTACGGGG
the LCS will be the length of the first string.

If we have our initial string : TTCTACGGGG
and we delete an existing character anywhere in the string : TTCTACGGG
the LCS will be the length of the second string.

If we have our initial string : TTCTACGGGGG
and we change a character anywhere in our string: GTCTACGGGGG
the LCS will be the lenght of either string -1. 

Adding and deleting a character functions similary to changing a character, as the LCS will decrease by 1 but the length of the second string is the same as the original. This complicated matters, as we don't know which operation actually caused the change we see.

Thus, we will find the minium number of operations possible to make the difference. We will assume that every change we see is the simplest form of the change, meaning that if we see an additional T, we assume that it was added as a T, and not added as G, deleted, added as C, changed to A, then changed to T. Thus, each difference we see will be counted as 1 modification. This assumption may not be entirely accurate, as some changes most likely were not the simplest. However, we assume that the actual amount of changes of each string is proportional to the minimum possible, that is, if the difference between two strings is a minimum of 10 operations and another two strings is a minimum of 20 operations, we assume that there were roughly twice as many overall operations in the second set of strings. Finding the minimum number of changes possible gives us a consistent and objective metric to compare different family trees.

## LCS

We must first calculate the LCS between all combinations of strings

In [99]:
def lcs(X , Y):
    # get the string lenghts and create an array
    m,n = len(X),len(Y)
    array = [[None]*(n+1) for i in range(m+1)]
     
    #iterate through all positions in array
    for i in range(m+1):
        for j in range(n+1):
            # if either is 0, the LCS is 0
            if i == 0 or j == 0 :
                array[i][j] = 0
            # if the added character is the same, the LCS is +1 
            elif X[i-1] == Y[j-1]:
                array[i][j] = array[i-1][j-1]+1
            # if not, the LCS is the max of the array 1 to the left and 1 above
            else:
                array[i][j] = max(array[i-1][j] , array[i][j-1])
    return array[m][n]

In [108]:
import numpy as np
import pandas as pd
import copy
import math
import itertools

num = len(strings)

lengths = pd.DataFrame(np.array([[None for i in range(num)]for j in range(num)]))
for col in range(num):
    for row in range(num):
        lengths[col][row] = lcs(strings[col][1],strings[row][1])
print ("LCS")
print (lengths)

LCS
     0   1   2   3   4   5   6
0  100  82  73  72  72  70  80
1   82  96  83  81  67  65  70
2   73  83  94  73  62  61  67
3   72  81  73  97  62  60  63
4   72  67  62  62  98  71  82
5   70  65  61  60  71  89  79
6   80  70  67  63  82  79  94


We can explore two strings and see what the LCS (and their lenghts) tells us. If we take string_0 and string_1, we know that the length of string_0 is 100 and the length of string_1 is 96. The LCS between the two is 82. If string_0 was the parent, we know that there must have been at least 4 deletions. Secondly, we know that there must have been 14 changed characters. Thus, at minimum there must have been 18 operations. If string_1 was the parent, we know that there must have been at least 4 additions, and that there must have been 14 changed characters, which again tells us 18 operations. Thus, the minimum number of changes needed if string_0 and string_1 had a direct relationship (regardless of parentage) is `max(len(string_0), len(string_0)) - LCS(string_0, string_1)`.

## Minimum Changes

This allows us to create a new table, which has the minimum number of changes between each combination of strings

In [109]:
min_changes = copy.deepcopy(lengths)
for col in range(num):
    for row in range(num):
        min_changes[col][row] = max(lengths[col][col],lengths[row][row]) - lengths[col][row]
print ("\nMinimum Changes")
print (min_changes)


Minimum Changes
    0   1   2   3   4   5   6
0   0  18  27  28  28  30  20
1  18   0  13  16  31  31  26
2  27  13   0  24  36  33  27
3  28  16  24   0  36  37  34
4  28  31  36  36   0  27  16
5  30  31  33  37  27   0  15
6  20  26  27  34  16  15   0


We now have a metric for assesing the position of two strings. This metric can be extended to family trees. A given family tree will have minimum number of changes for each relationship, and the sum of all of these will be the minimum number of changes for the entire family tree. We assume that these changes are unlikely, and thus the tree with the lowest minimum number of changes is the most likely to occur.

We can now examine this table in order to guess what the correct lineage should be. The first thing to look at is the lowest numbers, which indicate the most likely relationships.

We see that (0-1) has 18 changes, (0-6) has 20 changes, (1-2) has 13 changes, (1-3) has 16 changes, (4-6) has 16 changes and (5-6) has 15 changes.  These are all of the relationships with minimum changes of 20 or less, while other relationships have 24-37 changes. From this initial info, it is natural to conclude, that 0 is the grandfather with children 1 and 6, 1 has children 2 and 3, and 6 has children 4 and 5. All of the most likely relationships are included, and so we can confidently say that this is the most likely lineage.

# Probability of Mistake

We are now tasked with estimating the probability of insertions, deletions, and mutations. We can first make the assumption that for each character in the string, there is a small likihood of one of the three transformations. It is unclear if this is the actual process, but it makes sense given the information we have. We can define the probability of a single transformation as $P$. For a given string, the probability of making k mistakes, given n characters is $P_{k-mistakes} = \binom{n}{k} P^ k(1-P)^{n-k} $. 

Using a large set of data, we could get empirical estimates of $P_{k-mistakes}$ for given k values. We could then solve for $P$. We could first start by knowing a specific probability, like $P_{10-mistakes}$. We could use this to solve for $P$. However, if we did not have a large enough data set, we might get a different value of $P$ if we substituted in $P_{15-mistakes}$. In this scenario, we could find a $P$ value which creates $P_{k-mistakes}$ that are closest to the experimental $P_{k-mistakes}$ values we have (using some metric, like minimizing error).

The question of finding the probability of specific mistake is more complicated. The reason being, it is difficult to determine if a specific mistake happened and not another. If we had a string CGT and another string CAT, we can't simply say that there was a mutation. It could be that mutations are acutally incredibly unlikely, and instead deletion and insertion create many of the signs we think come from mutations. Thus, with this dataset, the options for determining the probability for individual types of mistakes are limited.

# General Algorithm

While with this example, we were able to manually examine the strings and determine which lineage was the most likely. However, this would not be possible with longer strings or more strings. Instead, we should devise a general algorithm that tells us the most likely lineage. What we will do is run a parameter sweep of all possible lineages, and determine which is optimal (this is a brute force approach).

With this algorithim, I will assume that there is a complete tree, i.e. that the number of nodes n = 2^g for some g. It returns the mininum possible changes and the associated tree.

In [105]:
def tree_checker(tree, min_changes):
    """
    tree - is a tuple of the strings in the specific tree order.
        for example if lis = (0,1,2,3,4,5,6),
        the grandparent is 0 with children 1 and 2, 
        1's children are 3 and 4 and 2's children are 5 and 6
        
    min_changes - this is the min_changes array we got in the last section
    
    """
    gen = math.log(len(tree),2)
    answer = 0
    index = 0
    for i in range(2**(int(gen))-1):
        for _ in range(2):
            index +=1
            answer += min_changes[tree[i]][tree[index]]
    return answer
                

answer_key = []
parameter_sweep = list(itertools.permutations(range(7)))
for parameter in parameter_sweep:
    answer_key.append(tree_checker(parameter, min_changes))

min_val = min(answer_key)
answer_key = np.array(answer_key)
indexies = np.where(answer_key == min_val)
print ("The minimum number of total changes for any lineage is", min_val)
print ("This is achieved with the grandfather as string", parameter_sweep[indexies[0][0]][0])
print ("with children ", parameter_sweep[indexies[0][0]][1], 'and', parameter_sweep[indexies[0][0]][2])
print ("whose children are",parameter_sweep[indexies[0][0]][3] ,parameter_sweep[indexies[0][0]][4])
print ('and ',parameter_sweep[indexies[0][0]][5] ,parameter_sweep[indexies[0][0]][6], ", respectively.")


The minimum number of total changes for any lineage is 98
This is achieved with the grandfather as node 0
with children  1 and 6
whose children are 2 3
and  4 5 , respectively.


This gave us the same answer as what we got when we manually checked the strings. This algorithm will give us the correct answer every single time, as it checks all possible parameters. It is useful in this sense, as we know it will always be correct. However, as we will soon see, with large sets of genes, it would take a prohibitively long amount of time.

## Complexity

We now must examine the complexity of our algorithm. We will define M as the length of a gene (while not all genes are the same, they are similar), and N as the number of genes. We first note that the complexity of our LCS algorithm is O(M^2), as it creates a table with width and length proportional to the length of a gene. We then do this N^2 times, as we must compute the LCS for each pair of strings. Thus, getting our minimum_changes array is O(M^2 N^2). We must now find the complexity of our tree checker parameter sweep. This operation is O(N!). 

This complexity is quite bad, as with any large number of genes, this would be impossible to compute.


## Greedy 

As the time complexity of our brute force algorithm did not scale well, I created a greedy algorithm that attempts to solve the problem. Note that this algorithm is not guaranteed to find the optimal solution. 

The first thing to do is to find the root node. In my algorithm, I found the node that is most similar to all other nodes (least changes to get to all other nodes). With this done, the next step is to find the children. Note that my specific algorithm does not take advantage of any optimal substructure. With each parent, I find the children using a greedy approach. It was initially appealing to have a recursive function, where after a child is found, and its children need to be found, a recursive function is called to solve a similar subproblem. The only problem with this is that the available strings (strings not already in the tree) are very important, and as such the order of solving problems matters. I chose to have two greedy choices for each parent (the strings with the lowest minimum changes). Each of these strings is added to a `waiting_for_kids` list and taken of of the `available` list. When there are no more strings to add the process is done. The greedy aspect of this algorithm is choosing children, as the locally optimal choice is always made, despite the future consequences.

In [146]:

def greedy(min_changes):
    #find the column with the most similarity
    root = min_changes.sum(axis=0).argmin()
    
    #create the lists
    tree = [root]
    waiting_for_kids = [root]
    available = list(range(len(min_changes)))
    available.remove(root)
    
    while available:
        active = waiting_for_kids.pop(0)
        
        #find child one
        child1 = choose_children(min_changes, active, available)
        available.remove(child1)
        tree.append(child1)
        waiting_for_kids.append(child1)
        
        #find child two
        child2 = choose_children(min_changes, active, available)
        available.remove(child2)
        tree.append(child2)
        waiting_for_kids.append(child2)
    return tree
    

    #this function is greedy, choosing the child with the lowest minimum changes
def choose_children(min_changes, active, available):
    lis = []
    counter = -1
    for i in available:
        counter +=1
        lis.append(min_changes[active][i])
        if lis[counter] == min(lis):
            index = i
    return index
        
#choose_children(min_changes, 1, available)

sol = greedy(min_changes)
print (sol)
print (tree_checker(sol, min_changes))

[1, 2, 3, 6, 0, 4, 5]
156


As can be seen, this algorithm does not achieve the optimal solution. We do, however, get a better running time. An upper bound time complexity of this algorithm is O(N^2). However, in order to get our `min_changes`, we have an O(M^2 N^2) algorithm. This absorbs the O(N^2), and our overall running time for computing the lineage is O(M^2 N^2).

The first problem it has is choosing the root node, as it chooses the incorrect node. Secondly, the greedy aspect, while increasing speed, also leads to suboptimal solutions. With real genome sequencing, we have a large interest in being correct, and thus we would need to find a better algorithm.

